# Convert the SWMM model values of DWF to the WEST model 

In [1]:

import SWMMToWESTConvert.aggregateNetwork as agN
import SWMMToWESTConvert.updateWESTfiles as uw


In [2]:
#MODEL points for checking the flow
LIMOILOU_COL= "RA_180174" #"DOM_941798" pipe
PASCAL_COL = "U89A" #"U004" pipe
STSACRA_COL = "R689587" #"UNI_602608" pipe
BEAU_COL = 'Pompe-Beauport'
NOEST_COL= "R016595" #'U003_Nord_Ouest' pipe
linkMeasurementFlow = [LIMOILOU_COL,BEAU_COL,STSACRA_COL,NOEST_COL,PASCAL_COL] #Change this to the nodes within the path

WTP_TANK = "RA_606859"
ST_SACR_LEAF_NODE= "R007637"



# Get pipe sections from detailed model ------------------------

In [3]:
file = 'DWF2022.inp' 

#Links measurement are assumed to be discharging into the trunk path (not in it)
pipesModel, catchmentsModel = agN.aggregateAndModelNetwork(file, WTP_TANK, linkMeasurementFlow, ST_SACR_LEAF_NODE)

Number of nodes  1493 , outlets  1421 , startPoints  72 , and after cleaned  70
There were 17 subcatchments with area 0
There were 0 DWFs with inflow type different than FLOW
There were 0 DFs with other constituents than FLOW
There are 21 connections to the path
7  relevant branches
9  connections to the path to be converted into catchments
4  points to break the path were found
Number of divisions by meassurement or input nodes 5


c:\Users\lllov\OneDrive - Université Laval\Desktop\01-Repositories\SWMMtoWEST\.venv\lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


ValueError: columns overlap but no suffix specified: Index(['OutletNode'], dtype='object')

In [ ]:
file = 'DWF2022.inp' 

pipesModel, catchmentsModel = agN.aggregateTrunk(file, WTP_TANK, ST_SACR_LEAF_NODE, linkMeasurementFlow)

There were  17 subcatchments with area 0
Number of nodes  1493 , outlets  1421 , startPoints  72 , and after cleaned  70
R008120   pop from empty list
R571863   pop from empty list
There are 21 connections to the path
5 connections to the path were removed due to no flow at anytime.
Number of measurement points 5
Number of catchments nodes  118
Number of dw flows  91
Number of direct flows  133
Number of divisions by meassurement or input nodes 17
DOM_7925 - DOM_602606
D: 0.9
Len: 754.429
Slope: 0.001949819002185764
UNI_602607 - UNI_8693
D: 0.9
Len: 254.241
Slope: 0.0017699741583772847
UNI_8674 - UNI_13606
D: 0.9
Len: 613.9000000000001
Slope: 0.0016843134060921955
UNI_13117 - UNI_13121
D: 0.6
Len: 125.44699999999999
Slope: 0.01457189091807696
UNI_1568728 - UNI_12850
D: 0.75
Len: 275.34700000000004
Slope: 0.003787947571609642
UNI_1635183 - C21
D: 0.75
Len: 302.15599999999995
Slope: 0.006586001932776452
UNI_19439 - UNI_20122
D: 1.2
Len: 501.327
Slope: 0.000931527725416745
UNI_1208035 - U

c:\Users\lllov\OneDrive - Université Laval\Desktop\01-Repositories\SWMMtoWEST\.venv\lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [4]:
catchmentsModel 

[{'PipeName': 'DOM_7925 - DOM_602606',
  'AreaCatchment': 8880.0,
  'NumberPeople': 3068,
  'FlowPerPerson': 0.4,
  'TimePattern': ['0.35',
   '0.249',
   '0.174',
   '0.176',
   '0.178',
   '0.375',
   '1.137',
   '1.5',
   '1.731',
   '1.779',
   '1.627',
   '1.41',
   '1.245',
   '1.008',
   '0.889',
   '0.997',
   '1.285',
   '1.437',
   '1.406',
   '1.291',
   '1.207',
   '1.176',
   '0.847',
   '0.525'],
  'DirectFBaseline': 1719.3600000000001,
  'EndNode': False},
 {'PipeName': 'DOM_7925 - DOM_602606[input]',
  'AreaCatchment': 0,
  'DirectFBaseline': 0,
  'NumberPeople': 16546,
  'TimePattern': ['0.8349720013081973',
   '0.8071398009804694',
   '0.7871290914266064',
   '0.785889500024899',
   '0.7864082550720318',
   '0.8332161512003997',
   '1.018823947857598',
   '1.1220453172287892',
   '1.1852048349064956',
   '1.201410445612298',
   '1.1659484729320828',
   '1.1110054632019908',
   '1.0672165342154196',
   '1.0072904672445222',
   '0.9738072940681994',
   '0.99681205544793

In [5]:
pipesModel

[{'PipeName': 'DOM_7925 - DOM_602606',
  'AreaTank': 169.746525,
  'Volmax': 97.77399840000002,
  'k': 0.0005448999045155582,
  'tanksIndexes': [1, 2, 3, 4]},
 {'PipeName': 'UNI_602607 - UNI_8693',
  'AreaTank': 228.8169,
  'Volmax': 131.79853440000002,
  'k': 0.000771367244997673,
  'tanksIndexes': [5]},
 {'PipeName': 'UNI_8674 - UNI_13606',
  'AreaTank': 184.17000000000002,
  'Volmax': 106.08192000000004,
  'k': 0.0006366387436501568,
  'tanksIndexes': [6, 7, 8]},
 {'PipeName': 'UNI_13117 - UNI_13121',
  'AreaTank': 9.408525,
  'Volmax': 3.6128736,
  'k': 2.113423322069251e-05,
  'tanksIndexes': [9, 10, 11, 12, 13, 14, 15, 16]},
 {'PipeName': 'UNI_1568728 - UNI_12850',
  'AreaTank': 68.83675000000001,
  'Volmax': 33.04164,
  'k': 0.0002122816356344216,
  'tanksIndexes': [17, 18, 19]},
 {'PipeName': 'UNI_1635183 - C21',
  'AreaTank': 32.37385714285713,
  'Volmax': 15.539451428571423,
  'k': 7.554330076568203e-05,
  'tanksIndexes': [20, 21, 22, 23, 24, 25, 26]},
 {'PipeName': 'UNI_1943

In [9]:
for p in pipesModel:

    print(p['PipeName'], p['tanksIndexes'][-1])

DOM_7925 - DOM_602606 4
UNI_602607 - UNI_8693 5
UNI_8674 - UNI_13606 8
UNI_13117 - UNI_13121 16
UNI_1568728 - UNI_12850 19
UNI_1635183 - C21 26
UNI_19439 - UNI_20122 27
UNI_1208035 - UNI_1208035 28
UNI_1208036 - UNI_1648876 29
UNI_18294 - UNI_19917 30
UNI_19920 - UNI_18252 31
UNI_18251 - UNI_18251 32
DOM_18248 - DOM_608354 33
DOM_608355 - DOM_19706 34
DOM_15610 - DOM_15368 35
DOM_14828 - DOM_20129 36
DOM_14831 - DOM_15857 37
DOM_19723 - DOM_24273 38
DOM_20724 - DOM_1568036 39
DOM_20726 - DOM_20726 40
DOM_23811 - DOM_23811 41
DOM_23809 - DOM_35983 44
DOM_1631422 - DOM_1631422 45
DOM_1640876 - DOM_1631424 46


# Start modifying WEST MODEL --------------------------

In [6]:
#Variables that can be changed accordingly

#classes to be assigned to the west blocks by model
SEWER_CLASS = 'psvdmSewerTank_Classplus_decay'
CATCH_CLASS = 'Catchment_NoRetention_IndirectStormwater'
CONNECTOR_CLASS = 'Kosim1ToPSVDM_curve_tracerflow_ClassplusViral'


In [7]:
#INFILTRATION = 'Infiltracion'

connProps = []
connProps.append(18)
connProps.append([23.6,47.2,65.8,141.5,283,565.9,800,943.2,1800,2829.6])

#the last section , the propertie tank index, the last index props[-1][TANK_INDEXES][-1]
uw.setPropertiesAndClasses('01-Data/05-NewWESTModel46Sections/Project46Sections.Layout.xml',
                     '02-Output/03-NewWEST46Sections/Project46SectionsMod.Layout.xml',
                     SEWER_CLASS,pipesModel,
                     CATCH_CLASS,catchmentsModel,
                     CONNECTOR_CLASS,connProps)




FileNotFoundError: [Errno 2] No such file or directory: '01-Data/05-NewWESTModel46Sections/Project46Sections.Layout.xml'

# Find trunk pipe from entire network

In [3]:
import SWMMToWESTConvert.findPaths as fp
import SWMMToWESTConvert.getNetworkFromSWMM as gnfs

file = 'DWF2022.inp' 

linksN = gnfs.getsNetworksLinks(file)

outFile = gnfs.getSimulationResultsFile(file)

trunkCalculatedDF= fp.findMainFlowPath(WTP_TANK,outFile,linksN)

TESTS!!

In [1]:

import SWMMToWESTConvert.getNetworkFromSWMM as gnfs
import swmmio

file = 'DWF2022.inp' 

model = swmmio.Model(file)

dflows, dflowsTimeseries  = gnfs.getWaterDirectFlows(model)

display(dflows)
display(dflowsTimeseries)

There were  0 DFs with other constituents than FLOW


,Time Series,Sfactor,Baseline
Node,,,
1_144,NaN,1,0.003000
1_156,NaN,1,0.003453
1_198,NaN,1,0.001100
1_210,NaN,1,0.009281
1_218,NaN,1,0.005200
...,...,...,...
RA_11071,NaN,1,0.000001
RA_14614,NaN,1,0.000001
RA_207381,NaN,1,0.000001


,Date,Time,Value
Name,,,


In [4]:
# Instantiate a swmmio model object
import SWMMToWESTConvert.getNetworkFromSWMM as gnfs
import swmmio

file = 'DWF2022.inp' 

model = swmmio.Model(file)
    
links = gnfs.getsNetworksLinks(file)

gnfs.getNodesLeaves(model, links)

Number of nodes  1493 , outlets  1421 , startPoints  72 , and after cleaned  70


Name
E-12-85        E-12-85
J12                J12
J31                J31
J43                J43
J64                J64
               ...    
R673150        R673150
R673737        R673737
R690064        R690064
RA_10563      RA_10563
RA_380483    RA_380483
Name: Name, Length: 70, dtype: object